In [1]:
import pandas as pd
import datetime as dt

# Define the start and end dates
start = dt.datetime(2021, 7, 1)
end = dt.datetime(2021, 12, 28)

# Load the CSV file and set 'Date' as the index
data = pd.read_csv('Tatacoffee13_21.csv', parse_dates=['Date'], index_col='Date')

# Filter the data based on the index
stk_data = data[(data.index >= start) & (data.index <= end)]

# Display the filtered data
print(stk_data)

              Open    High     Low   Close
Date                                      
2021-07-01  188.60  189.00  183.00  184.55
2021-07-02  183.05  187.50  182.15  186.40
2021-07-05  187.85  198.35  187.85  192.45
2021-07-06  193.80  194.80  187.35  188.80
2021-07-07  187.80  193.95  185.40  192.85
...            ...     ...     ...     ...
2021-12-22  202.90  207.80  201.35  205.00
2021-12-23  206.00  206.85  202.05  202.95
2021-12-24  203.90  203.90  199.35  201.00
2021-12-27  200.00  222.00  196.00  218.35
2021-12-28  219.65  220.45  211.55  212.35

[123 rows x 4 columns]


In [2]:

stk_data=stk_data[["Open","High","Low","Close"]]

In [3]:
stk_data

,Open,High,Low,Close
Date,,,,
2021-07-01,188.60,189.00,183.00,184.55
2021-07-02,183.05,187.50,182.15,186.40
2021-07-05,187.85,198.35,187.85,192.45
2021-07-06,193.80,194.80,187.35,188.80
2021-07-07,187.80,193.95,185.40,192.85
...,...,...,...,...
2021-12-22,202.90,207.80,201.35,205.00
2021-12-23,206.00,206.85,202.05,202.95
2021-12-24,203.90,203.90,199.35,201.00


In [4]:
stk_data.index

DatetimeIndex(['2021-07-01', '2021-07-02', '2021-07-05', '2021-07-06',
               '2021-07-07', '2021-07-08', '2021-07-09', '2021-07-12',
               '2021-07-13', '2021-07-14',
               ...
               '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-20',
               '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24',
               '2021-12-27', '2021-12-28'],
              dtype='datetime64[ns]', name='Date', length=123, freq=None)

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (123, 4)


In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
data1

,Open,High,Low,Close
0,0.111864,0.022422,0.083571,0.052680
1,0.017797,0.000000,0.067426,0.086876
2,0.099153,0.162182,0.175689,0.198706
3,0.200000,0.109118,0.166192,0.131238
4,0.098305,0.096413,0.129155,0.206100
...,...,...,...,...
118,0.354237,0.303438,0.432099,0.430684
119,0.406780,0.289238,0.445394,0.392791
120,0.371186,0.245142,0.394112,0.356747
121,0.305085,0.515695,0.330484,0.677449


In [7]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

98
X_train length: (98, 4)
X_test length: (25, 4)
y_train length: (98, 4)
y_test length: (25, 4)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [10]:
performance

{'Model': [], 'RMSE': [], 'MaPe': [], 'Lag': [], 'Test': []}

In [ ]:
def combination(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [14]:
def combination(dataset, listt):
    print(listt)
    datasetTwo = dataset[listt]
    test_obs = 28
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    
    from statsmodels.tsa.api import VAR
    # Fit the VAR model for different lags (1 to 10)
    for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    
    # Select optimal order based on AIC
    x = model.select_order(maxlags=12)
    order = x.selected_orders["aic"]
    
    # Fit the model with the selected order
    result = model.fit(order)
    
    # Get the lagged values (last 'order' rows)
    lagged_Values = train.values[-order:]
    
    # Generate the forecast for the next 28 periods
    pred = result.forecast(y=lagged_Values, steps=28) 
    
    # Convert the forecasted array to a DataFrame with column names
    preds = pd.DataFrame(pred, columns=listt)
    
    # If you used MinMaxScaler, you can inverse transform the scaled values
    pred_original = Ms.inverse_transform(pred)
    pred_original_df = pd.DataFrame(pred_original, columns=listt)
    
    # Print the forecasted values (scaled and original)
    print("Scaled Forecasted Values:")
    print(preds.head())
    
    print("\nOriginal Forecasted Values (Inverse Transformed):")
    print(pred_original_df.head())
    
    # Save the forecasted values to CSV
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    
    # Evaluate the model (RMSE and MAPE)
    from sklearn.metrics import mean_squared_error
    rmse = round(mean_squared_error(test, pred, squared=False))
    
    from sklearn.metrics import mean_absolute_percentage_error
    mape = mean_absolute_percentage_error(test, pred)
    
    # Store the performance metrics
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    
    # Convert the performance dictionary to a DataFrame
    perf = pd.DataFrame(performance)
    
    return perf, result, preds, pred_original_df

In [15]:
# Define the list of columns you want to use in the VAR model
listt = ["Open", "High", "Low", "Close"]

# Now call the combination function
perf, result, pred,pred_original_df= combination(data1, listt)

['Open', 'High', 'Low', 'Close']
Order = 1
AIC:  -24.773550390390394
BIC:  -24.232423840971244

Order = 2
AIC:  -24.537520362684866
BIC:  -23.557159268561023

Order = 3
AIC:  -24.357551846059245
BIC:  -22.93219308511848

Order = 4
AIC:  -24.169233695339997
BIC:  -22.292987031129606

Order = 5
AIC:  -24.06310560350932
BIC:  -21.729949911201075

Order = 6
AIC:  -23.914905847924874
BIC:  -21.1186852077764

Order = 7
AIC:  -23.7209251632359
BIC:  -20.455344816878263

Order = 8
AIC:  -23.535156866565718
BIC:  -19.79377903136566

Order = 9
AIC:  -23.366103168399235
BIC:  -19.142342705079244

Order = 10
AIC:  -23.4111233012541
BIC:  -18.69824322990808

Scaled Forecasted Values:
       Open      High       Low     Close
0  0.823819  0.689793  0.901025  0.849688
1  0.796399  0.672465  0.861075  0.824173
2  0.772578  0.655263  0.830119  0.799596
3  0.749600  0.636684  0.803241  0.776267
4  0.727856  0.618308  0.779378  0.754724

Original Forecasted Values (Inverse Transformed):
         Open    

In [16]:
pred_original_df

,Open,High,Low,Close
0,230.605293,233.647150,226.038970,227.668098
1,228.987555,232.487906,223.935600,226.287770
2,227.582121,231.337084,222.305750,224.958169
3,226.226398,230.094177,220.890662,223.696049
4,224.943493,228.864806,219.634273,222.530559
5,223.762183,227.709781,218.514094,221.470121
6,222.688896,226.649871,217.512963,220.512299
7,221.720169,225.688365,216.617023,219.650389
8,220.848769,224.821208,215.814657,218.876284
9,220.066291,224.041490,215.095827,218.181735


In [17]:
result